# Data import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from datetime import datetime, date
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [2]:
FILES_DIR = './files/'
Eco_df = pd.read_csv(FILES_DIR + 'Eco_df.csv')

In [3]:
session_id = [(list(set(x['item_no'].values))[0], '/'.join(x['session_id'].values)) for index, x in Eco_df.groupby('item_no')]
session_id_df = pd.DataFrame(data=session_id, columns=['item_no', 'session_id'])
session_id_df = session_id_df.set_index('item_no')

user_no = [(list(set(x['item_no'].values))[0], '/'.join(x['user_no'].values)) for index, x in Eco_df.groupby('item_no')]
user_no_df = pd.DataFrame(data=user_no, columns=['item_no', 'user_no'])
user_no_df = user_no_df.set_index('item_no')

total_df = pd.merge(session_id_df, user_no_df, on='item_no', how='inner')

In [14]:
label = LabelEncoder()
label.fit(total_df['session_id'])
total_df['session_id'] = label.transform(total_df['session_id'])

In [16]:
label = LabelEncoder()
label.fit(total_df['user_no'])
total_df['user_no'] = label.transform(total_df['user_no'])

In [ ]:
label = LabelEncoder()
label.fit(object_df[i])
object_df[i] = label.transform(object_df[i])

In [12]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()

dummy1 = label.fit(total_df['session_id']), label.transform(total_df['session_id']), pd.get_dummies(total_df['session_id'])      

        #  total_df['session_id'].str.get_dummies(sep=',') 


# dummy2 = label.fit(total_df['user_no']),
#          label.transform(total_df['user_no']),
#         #  total_df['user_no'].str.get_dummies(sep=',') 
#          pd.get_dummies(total_df['user_no'])   

In [13]:
dummy1

(LabelEncoder(),
 array([  919,  3584, 10242, ...,  7356,  7467,  7519]),
                           00036f34a8a43d56a24aeafce93ce4ee/3dd9015524e8e1d892d8576564e5904a  \
 item_no                                                                                       
 ++J/y7daNUk0gVvuZAyv3w==                                                  0                   
 ++KnGAHePrFDjX5hficy7w==                                                  0                   
 ++M/vQ2omwiRuFlSZk6Q+g==                                                  0                   
 ++PeNgz1/ylhHwdLJ6rzMQ==                                                  0                   
 +/DykVqeV8BTI3DLPuuTFg==                                                  0                   
 ...                                                                     ...                   
 zyQ0KZQ4WUK4e/IoYJz5Qg==                                                  0                   
 zz1zdZH7yh+YIWSkAlKOzA==                                     

# user

In [ ]:
train_df, test_df = train_test_split(Eco_df, test_size=0.25, random_state=42)

print(train_df.shape)
print(test_df.shape)

y_train = train_df['rating']
print(y_train.shape)

In [ ]:
X_train = pd.concat([pd.get_dummies(train_df['user_no'], prefix='user_no'),
        #    train_df['item_no'].apply(lambda x: dummy1.loc[x]),
           train_df['item_no'].apply(lambda x: dummy2.loc[x])], axis=1)

In [ ]:
import scipy
X_train_sparse = scipy.sparse.csr_matrix(X_train.values)

In [7]:
rating_matrix = X_train_sparse
# 아이템 기반 유사도 측정
rating_matrix_t = rating_matrix #np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy()#.fillna(0)
item_similarity = pd.DataFrame(cosine_similarity(matrix_dummy))

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean(np.power(np.array(y_true)-np.array(y_pred), 2)))

def score(model):
  id_pairs = zip(test_df['user_no'], test_df['item_no'])
  y_pred = np.array([model(user, item) for (user, item) in id_pairs])
  y_true = np.array(test_df['rating'])
  return RMSE(y_true, y_pred)

def CF_IBCF(user_no, item_no):
  if item_no in item_similarity:
    sim_scores = item_similarity[item_no]
    user_rating = rating_matrix_t[user_no]
    non_rating_idx = user_rating[user_rating.isnull()].index
    user_rating = user_rating.dropna()
    sim_scores = sim_scores.drop(non_rating_idx)
    mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
  else:
    mean_rating = 2.5
  
  return mean_rating
  
print('아이템 기반 협업 필터링 방식의 정확도:', score(CF_IBCF))

ValueError: Index contains duplicate entries, cannot reshape

# session